In [ ]:
import urllib.request
import json
import requests
from bs4 import BeautifulSoup
import datetime
import time
import pandas as pd
import xlrd as xl
import numpy
import os

In [ ]:
ano_inicio = 2004
#ano_inicio = 2021
now = datetime.datetime.now()
ano_actual = now.year

main_url = "https://wwwssl.aena.es/"

In [ ]:
url_AENA_IATA_aeropuertos = "https://raw.githubusercontent.com/abediaga/AENA/main/AENA_IATA_aeropuertos.json"
file_name = "AENA_IATA_aeropuertos.json"

urllib.request.urlretrieve(url_AENA_IATA_aeropuertos, file_name)

aeropuertos_ = {}

with open('AENA_IATA_aeropuertos.json') as f:
    aeropuertos = json.load(f)


for aero_key, aero_value in aeropuertos.items():
    #aeropuertos_[aero_key] = aero_value["Nombre"]
    aeropuertos_[aero_value["Nombre"]] = aero_key
    if "Otros" in aero_value.keys():
        #print(aero_value["Otros"])
        for item in aero_value["Otros"]:
            #print(item)
            #aeropuertos_[aero_key] = str(item)
            aeropuertos_[str(item)] = aero_key


In [ ]:
links_excels_AENA = {}

url_base = "https://www.aena.es"

for ano in range(ano_inicio,ano_actual+1):
    print(ano)
    url = "https://www.aena.es/es/estadisticas/informes-mensuales.html?anio=" + str(ano)
    print(url)
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    contenido = soup.find_all("div", {"class": "informes"})
    for elemento in contenido:
        if len(contenido) > 0:
            contenido2 = elemento.find_all("p", {"class": "h5 grueso bold"})
            if len(contenido2) > 0:
                mes = contenido2[0].text
                #print(mes)
            
            contenido3 = elemento.find_all("div", {"class": "descargas"})
            if len(contenido3) > 0:
                for elemento2 in contenido3:
                    contenido4 = elemento2.findAll("a")
                    if len(contenido4) > 0:
                        link = contenido4[1]['href']       
        #print("======")
        if ano in links_excels_AENA:
            links_excels_AENA[ano][mes] = url_base+link    
        else :
            links_excels_AENA[ano] = {}
            links_excels_AENA[ano][mes] = url_base+link
    

In [ ]:
index = []
for ano in links_excels_AENA:
    for mes in links_excels_AENA[ano]:
        tupla = (ano, mes)
        index.append(tupla)

df_index = pd.MultiIndex.from_tuples(index, names=('Año', 'Mes'))

df_AENA_pasajeros = pd.DataFrame(columns=[], index=df_index)
df_AENA_operaciones = pd.DataFrame(columns=[], index=df_index)
df_AENA_mercancias = pd.DataFrame(columns=[], index=df_index)

In [ ]:
for ano in links_excels_AENA:
    print(ano)
    for mes in links_excels_AENA[ano]:
        #print(mes)
        file = requests.get(links_excels_AENA[ano][mes])
        filename = 'aena_temp.xls'
        open(filename, 'wb').write(file.content)
        
        Workbook = xl.open_workbook(filename)
        sheets = Workbook.sheet_names()
        hoja_seleccionada = ""
        
        for sheet in sheets:
            if ("TRÁFICO" in sheet) or ("TRAFICO" in sheet):
                hoja_seleccionada = sheet
        if hoja_seleccionada == "":
                hoja_seleccionada = sheets[0]
        
        if (ano < 2009):
            df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5], usecols=[1,2,5,6,9,10], header=1)
        if (ano == 2009):
            if mes == "Enero":
                df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5,6,7], usecols=[2,4,7,8,11,12], header=1)
            else:
                df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5,6,7,8], usecols=[2,4,7,8,11,12], header=1)
        if (ano == 2010):
            df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5,6,7], usecols=[2,4,7,8,11,12], header=1)
        if (ano == 2011) or (ano == 2012):
            df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5,6], usecols=[2,4,7,8,11,12], header=1)
        if (ano == 2013):
            if mes == "Enero":
                df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5], usecols=[2,4,7,8,11,12], header=1)
            else:
                df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5,6,7], usecols=[2,4,7,8,11,12], header=1)
        if (ano > 2013) and (ano < 2021):
            df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5,6], usecols=[2,4,7,8,11,12], header=1)
        if (ano == 2021):
            if (mes == "Enero") or (mes == "Febrero"):
                df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5,6], usecols=[2,4,7,8,11,12], header=1)
            else:
                df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5,6], usecols=[2,3,7,8,12,13], header=1)
        if (ano > 2021):
            df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5,6], usecols=[2,3,7,8,11,12], header=1)
        
        df_pasajeros = df[df.columns[0:2]].copy()
        df_pasajeros = df_pasajeros.set_index(df_pasajeros.columns[0])
        df_pasajeros.index.name = 'Aeropuerto'
        df_pasajeros = df_pasajeros[df_pasajeros.index.notnull()]
        
        df_operaciones = df[df.columns[2:4]].copy()
        df_operaciones = df_operaciones.set_index(df_operaciones.columns[0])
        df_operaciones.index.name = 'Aeropuerto'
        df_operaciones = df_operaciones[df_operaciones.index.notnull()]
        
        df_mercancias = df[df.columns[4:6]].copy()
        df_mercancias = df_mercancias.set_index(df_mercancias.columns[0])
        df_mercancias.index.name = 'Aeropuerto'
        df_mercancias = df_mercancias[df_mercancias.index.notnull()]
        
        for index, row in df_pasajeros.iterrows():
            index = index.strip()
            if (index in aeropuertos_.keys()):
                if type(row.iloc[0]) == numpy.float64:
                    valor = int(row.iloc[0].astype(int))
                if type(row.iloc[0]) == float:
                    valor = int(row.iloc[0])
                if type(row.iloc[0]) == str:
                    valor = 0
                if type(row.iloc[0]) == int:
                    valor = int(row.iloc[0])
                df_AENA_pasajeros.loc[(ano,mes),aeropuertos_[index]] = valor
        
        for index, row in df_operaciones.iterrows():
            index = index.strip()
            if (index in aeropuertos_.keys()):
                #print(type(row.iloc[0]))
                if type(row.iloc[0]) == numpy.float64:
                    valor = int(row.iloc[0].astype(int))
                if type(row.iloc[0]) == float:
                    valor = int(row.iloc[0])
                if type(row.iloc[0]) == str:
                    valor = 0
                if type(row.iloc[0]) == int:
                    valor = int(row.iloc[0])
                df_AENA_operaciones.loc[(ano,mes),aeropuertos_[index]] = valor

        for index, row in df_mercancias.iterrows():
            index = index.strip()
            if (index in aeropuertos_.keys()):
                #print(type(row.iloc[0]))
                if type(row.iloc[0]) == numpy.float64:
                    valor = int(row.iloc[0].astype(int))
                if type(row.iloc[0]) == float:
                    valor = int(row.iloc[0])
                if type(row.iloc[0]) == str:
                    valor = 0
                if type(row.iloc[0]) == int:
                    valor = int(row.iloc[0])
                df_AENA_mercancias.loc[(ano,mes),aeropuertos_[index]] = valor        

In [ ]:
display(df_AENA_pasajeros)

In [ ]:
display(df_AENA_operaciones)

In [ ]:
display(df_AENA_mercancias)

In [ ]:
#DELETE 'aena_temp.xls'
os.remove("aena_temp.xls")

In [ ]:
#display(df_AENA_operaciones.loc[[2020,2021]])
#display(df_AENA_operaciones.index)
#display(df_AENA_pasajeros.loc[(2019,"Enero")])
#display(df_AENA_pasajeros)

In [ ]:
#Exportamos el dataframe de Mercabilbao a un Pickle
pd.to_pickle(df_AENA_pasajeros, "./raw_df_AENA_pasajeros.pkl")
pd.to_pickle(df_AENA_operaciones, "./raw_df_AENA_operaciones.pkl")
pd.to_pickle(df_AENA_mercancias, "./raw_df_AENA_mercancias.pkl")

In [ ]:
df_AENA_pasajeros.to_csv("pasajeros.csv")
df_AENA_operaciones.to_csv("operaciones.csv")
df_AENA_mercancias.to_csv("mercancias.csv")

In [ ]:
#display(list(df_AENA_pasajeros.columns))
display(list(df_AENA_pasajeros.index))
display(df_AENA_pasajeros["MAD"].loc[(2019,"Enero")])
display(df_AENA_operaciones["MAD"].loc[(2019,"Enero")])
display(int(df_AENA_mercancias["MAD"].loc[(2019,"Enero")]))